

> part of our codes are from:

> 1. TensorFlow Hub: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1
2. bert embedding: https://pypi.org/project/bert-embedding/
3. NLP-Models-Tensorflow: https://github.com/VihangaDehigaspitiya/NLP-Models-Tensorflow/blob/23213e7075fef72212bf4d81a9baa9db6c5e1305/README.md






In [0]:
#from sklearn.model_selection import train_test_split
#import pandas as pd
#import tensorflow as tf
#import tensorflow_hub as hub
#from datetime import datetime
#import os

In [13]:
!pip install bert-tensorflow

In [0]:
#!pip install tokenization

In [15]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
import collections

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

import pandas as pd
from datetime import datetime

TensorFlow is already loaded. Please restart the runtime to change versions.


In [17]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/1012 proj/")

In [0]:
dir = os.listdir("Data txt")
dir.sort()
os.chdir("/content/drive/My Drive/Colab Notebooks/1012 proj/Data txt")

In [0]:
df = pd.DataFrame(columns = ['origin','modern'])

for idx in range(len(dir)):
  filepath = dir[idx]
  if idx %2  == 0:
    data=pd.read_csv(filepath,delimiter="\t",header=None)
    data.columns = ['modern']
  else:
    trans=pd.read_csv(filepath,delimiter="\t",header=None)
    trans.columns = ['origin']
    trans['modern'] = data['modern']
    df=pd.concat([df, trans])

In [0]:
import pandas as pd
os.chdir("/content/drive/My Drive/Colab Notebooks/1012 proj/")
df = pd.read_csv('data.csv')

In [0]:
df

,origin,modern
0,I have a mind to strike thee ere thou speak’st.,I have half a mind to hit you before you speak...
1,"Yet if thou say Antony lives, is well, Or frie...","But if Antony is alive, healthy, friendly with..."
2,"Madam, he’s well.","Madam, he’s well."
3,Well said.,That’s well spoken.
4,And friends with Caesar.,And he’s friendly with Caesar.
...,...,...
21074,"How now, my metal of India?","How are you, my golden girl?"
21075,Get you all three into the boxtree.,"Hide behind the boxwood hedge, all three of you."
21076,Malvolio’s coming down this walk.,Malvolio’s coming down the path.
21077,He has been yonder i' the sun practising behav...,He’s been over there practicing how to act for...


In [21]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_repo'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 314.15 KiB | 3.57 MiB/s, done.
Resolving deltas: 100% (184/184), done.


AttributeError: ignored

In [0]:
tfe = tf.contrib.eager
tfe.enable_eager_execution(

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
#   multi_cased_L-12_H-768_A-12: cased multilingual
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL

LAYERS = [-1,-2,-3,-4]  # Take into account only last 4 layers
NUM_TPU_CORES = 8
MAX_SEQ_LENGTH = 128
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
BATCH_SIZE = 128
MAX_DIMENSION = 768  # 1024 for Large model


TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']

In [0]:
origin = df.iloc[:1000]['origin'].tolist()
modern = df.iloc[:1000]['modern'].tolist()

In [0]:
len(modern)

1000

In [0]:
!pip install bert-for-tf2

In [0]:
pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

# TF_KERAS must be added to environment variables in order to use TPU
os.environ['TF_KERAS'] = '1'

In [0]:
import tensorflow as tf
from bert import bert_tokenization as tokenization
import os
import random
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import numpy as np
import re

In [0]:
max_seq_length = 128
# Load Pre-Trained BERT Model via TF 2.0
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)


In [0]:
a = [1,2,3]
a[:2]

[1, 2]

In [0]:
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens) > max_seq_length:
      tokens = tokens[:128]
        #raise IndexError("Token length more than max seq length!")
    return [1] * len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens) > max_seq_length:
      tokens = tokens[:128]
      #  raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def get_ids(tokens, tokenizer, max_seq_length):
    if len(tokens) > max_seq_length:
      tokens = tokens[:128]
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length - len(token_ids))
    return input_ids

def batch_iter(id, mask, seg, label, batch_size, max_seq_length):
    """
        A mini-batch iterator to generate mini-batches for training neural network
        param data: a list of sentences. each sentence is a vector of integers
        param label: a list of labels
        param batch_size: the size of mini-batch
        param num_epochs: number of epochs
        return: a mini-batch iterator
        """
    assert len(id) == len(mask) == len(seg) == len(label)
    data_size = len(id)
    iter_num = data_size // batch_size # Avoid dimension disagreement

    for i in range(iter_num):
        start_index = i * batch_size
        end_index = start_index + batch_size

        ids = id[start_index: end_index]
        masks = mask[start_index: end_index]
        segs = seg[start_index: end_index]
        labels = label[start_index: end_index]
        #  print(len(labels))
            
        permutation = np.random.permutation(labels.shape[0])
        yield ids, masks, segs, labels

train_data_seq = []
train_data_word = []
train_label = []

def data_filter(origin, modern, total_size, tokenizer, max_seq_length):
    size = total_size // 2
    size = min([len(origin), len(modern), size])
    
    m_lines = [["[CLS]"] + tokenizer.tokenize(line) + ["[SEP]"] for line in origin]
    m_lines = [line for line in m_lines if len(line) < 128] #and len(line) > 2
    random.shuffle(m_lines)
    f_lines = [["[CLS]"] + tokenizer.tokenize(line) + ["[SEP]"] for line in modern]
    f_lines = [line for line in f_lines if len(line) < 128]  #and len(line) > 2
    random.shuffle(f_lines)

    m_lines = m_lines[: size]
    f_lines = f_lines[: size]
    
    lines = m_lines + f_lines
    label = np.append(np.zeros(len(m_lines)), np.ones(len(f_lines)))
    
    ids = [get_ids(token, tokenizer, max_seq_length) for token in lines]
    masks = [get_masks(token, max_seq_length) for token in lines]
    segs = [get_segments(token, max_seq_length) for token in lines]
    
    # Shuffle
    perm = np.random.permutation(len(lines))
    ids = np.array(ids)
    masks = np.array(masks)
    segs = np.array(segs)
    ids = ids[perm, :]
    masks = masks[perm, :]
    segs = segs[perm, :]
    label = label[perm]
    
    return ids, masks, segs, label
    
# Parameters
total_size = 3000
batch_size = 128

ids, masks, segs, labels = data_filter(origin, modern, total_size, tokenizer, max_seq_length)
train_data = batch_iter(ids, masks, segs, labels, batch_size, max_seq_length)

for i, train_input in enumerate(train_data):
    print("No.", i, "iteration")
    input_id, input_mask, input_seg, label = train_input
    seq_data, word_data = model.predict([input_id, input_mask, input_seg])
    train_data_seq.extend(seq_data)
    train_data_word.extend(word_data)
    train_label.extend(label)
    print(len(train_label))
    print(len(train_data_seq), len(train_data_seq[0]))
    print(len(train_data_word), len(train_data_word[0]))
    
train_data_seq = np.array(train_data_seq)
train_data_word = np.array(train_data_word)
train_label = np.array(train_label)
np.save("data_seq.npy", train_data_seq)
np.save("data_word.npy", train_data_word)
np.save("label.npy", train_label)

No. 0 iteration
128
128 1024
128 128
No. 1 iteration
256
256 1024
256 128
No. 2 iteration
384
384 1024
384 128
No. 3 iteration
512
512 1024
512 128
No. 4 iteration
640
640 1024
640 128
No. 5 iteration
768
768 1024
768 128
No. 6 iteration
896
896 1024
896 128
No. 7 iteration
1024
1024 1024
1024 128
No. 8 iteration
1152
1152 1024
1152 128
No. 9 iteration
1280
1280 1024
1280 128
No. 10 iteration
1408
1408 1024
1408 128
No. 11 iteration
1536
1536 1024
1536 128
No. 12 iteration
1664
1664 1024
1664 128
No. 13 iteration
1792
1792 1024
1792 128
No. 14 iteration
1920
1920 1024
1920 128


In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/1012 proj/")
np.save("data_seq.npy", train_data_seq)
np.save("data_word.npy", train_data_word)
np.save("label.npy", train_label)

# **new section**

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
from keras.utils.np_utils import to_categorical
from bert import bert_tokenization as tokenization
import pandas as pd
import numpy as np
import json
import os

max_seq_len = 128
n_hidden_unit = 128

# Refer to https://towardsdatascience.com/bert-in-keras-with-tensorflow-hub-76bcbc9417b
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

#with open("female2male_part1.txt", "r") as f:
 # questions = f.readlines()
  #questions = [q.lower().strip() for q in questions]

#with open("female2male_part2.txt", "r") as f:
 # answers = f.readlines()
  #answers = [a.lower().strip() for a in answers]
origin = df.iloc[:1000]['origin'].tolist()
modern = df.iloc[:1000]['modern'].tolist()
outdir = "./"

Using TensorFlow backend.


In [0]:
# Shuffle
arr_origin = np.array(origin)
arr_modern = np.array(modern)
perm = np.random.permutation(len(modern))
shuffled_input = arr_origin[perm]
shuffled_label = arr_modern[perm]

def build_vocab_dict(tokenizer, questions, answers):
    vocab = {}
    for sample in zip(questions, answers):
        question = ["[CLS]"] + tokenizer.tokenize(sample[0]) + ["[SEP]"]
        answer = ["[CLS]"] + tokenizer.tokenize(sample[1]) + ["[SEP]"]

        for q in question:
            vocab[q] = len(vocab) if q not in vocab else vocab[q]
        for a in answer:
            vocab[a] = len(vocab) if a not in vocab else vocab[a]
    vocab["[PAD]"] = len(vocab)
    vocab_idx2word = {vocab[key] : key for key in vocab.keys()}

    return vocab, vocab_idx2word

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens) > max_seq_length:
        tokens = tokens[:max_seq_length]
        #raise IndexError("Token length more than max seq length!")
    return [1] * len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens) > max_seq_length:
        tokens = tokens[:max_seq_length]
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length - len(token_ids))
    return input_ids

def batch_iter(encode_idt, encode_mask, encode_seg, decode_idt, decode_mask, decode_seg, label, batch_size, max_seq_length, vocab_size):
    # assert len(idt) == len(mask) == len(seg) == len(label)
    data_size = len(encode_idt)
    iter_num = data_size // batch_size # Avoid dimension disagreement

    for i in range(iter_num):
        start_index = i * batch_size
        end_index = start_index + batch_size

        encode_ids = encode_idt[start_index: end_index]
        encode_masks = encode_mask[start_index: end_index]
        encode_segs = encode_seg[start_index: end_index]

        decode_ids = decode_idt[start_index: end_index]
        decode_masks = decode_mask[start_index: end_index]
        decode_segs = decode_seg[start_index: end_index]

        labels = label[start_index: end_index]
        print(encode_ids.shape)        
        yield [encode_ids, encode_masks, encode_segs, decode_ids, decode_masks, decode_segs], to_categorical(labels, vocab_size)

def prepare_data(origin, modern, vocab, tokenizer, max_seq_length):
    data = [["[CLS]"] + tokenizer.tokenize(line) + ["[SEP]"] for line in origin]
    label = [["[CLS]"] + tokenizer.tokenize(line) + ["[SEP]"] for line in modern]

    # Generate word length for LSTM model
    d_length = [len(d) for d in data]
    l_length = [len(l) for l in label]
    max_length = max([max(d_length), max(l_length)])

    # Expand training dataset based on max length of sentence
    for i in range(len(data)):
        length = len(data[i])
        if length < max_length:
            data[i] += ["[PAD]"] * (max_length - length)

    for i in range(len(label)):
        length = len(label[i])
        if length < max_length:
            label[i] += ["[PAD]"] * (max_length - length)

    encode_ids = [get_ids(token, tokenizer, max_seq_length) for token in data]
    encode_masks = [get_masks(token, max_seq_length) for token in data]
    encode_segs = [get_segments(token, max_seq_length) for token in data]

    decode_ids = [get_ids(token, tokenizer, max_seq_length) for token in label]
    decode_masks = [get_masks(token, max_seq_length) for token in label]
    decode_segs = [get_segments(token, max_seq_length) for token in label]

    labels = []
    for tag in label:
        tags = [vocab[idx] for idx in tag]
        labels.append(tags)
        
    return encode_ids, encode_masks, encode_segs, decode_ids, decode_masks, decode_segs, labels

tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
if os.path.isfile('vocab_w2i.json') and os.path.isfile('vocab_i2w.json'):
    with open('vocab_w2i.json', 'r') as f:
        vocab_w2i = json.load(f)
    with open('vocab_i2w.json', 'r') as f:
        vocab_i2w = json.load(f)

else:
    vocab_w2i, vocab_i2w = build_vocab_dict(tokenizer, origin, modern)
    with open('vocab_w2i.json', 'w') as f:
        json.dump(vocab_w2i, f)
    with open('vocab_i2w.json', 'w') as f:
        json.dump(vocab_i2w, f)

vocab_size = len(vocab_i2w)
print(vocab_size)

2775


In [0]:
encode_ids, encode_masks, encode_segs, decode_ids, decode_masks, decode_segs, labels = prepare_data(shuffled_input, shuffled_label, vocab_w2i, tokenizer, max_seq_len)

encode_ids = np.array(encode_ids)
encode_masks = np.array(encode_masks)
encode_segs = np.array(encode_segs)
decode_ids = np.array(decode_ids)
decode_masks = np.array(decode_masks)
decode_segs = np.array(decode_segs)
labels = np.array(labels)

# Shuffle
perm = np.random.permutation(encode_ids.shape[0])
encode_ids = encode_ids[perm, :]
encode_masks = encode_masks[perm, :]
encode_segs = encode_segs[perm, :]
decode_ids = decode_ids[perm, :]
decode_masks = decode_masks[perm, :]
decode_segs = decode_segs[perm, :]
labels = labels[perm, :]

# Split dataset
size = len(shuffled_label)
unit = int(size / 5)

train_encode_ids = encode_ids[: unit * 4, :]
train_encode_masks = encode_masks[: unit * 4, :]
train_encode_segs = encode_segs[: unit * 4, :]
test_encode_ids = encode_ids[unit * 4 : , :]
test_encode_masks = encode_masks[unit * 4 : , :]
test_encode_segs = encode_segs[unit * 4 : , :]
train_decode_ids = decode_ids[: unit * 4, :]
train_decode_masks = decode_masks[: unit * 4, :]
train_decode_segs = decode_segs[: unit * 4, :]
test_decode_ids = decode_ids[unit * 4 : , :]
test_decode_masks = decode_masks[unit * 4 : , :]
test_decode_segs = decode_segs[unit * 4 : , :]

train_labels = labels[: unit * 4, 1 :]
test_labels = labels[unit * 4 : , 1 :]

def seq2seq_model(vocab_size, max_seq_length=128, n_hidden_unit=128):
  # Encoder
  encode_id = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="encode_input_word_ids")
  encode_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                      name="encode_input_mask")
  encode_seg_id = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, 
                                        name="encode_input_segment_ids")
  
  encode_input = [encode_id, encode_mask, encode_seg_id]
  _, encode_embedding = bert_layer(encode_input)
  encode_LSTM = tf.keras.layers.LSTM(n_hidden_unit, return_state=True)
  encode_output, state_h, state_c = encode_LSTM(encode_embedding)
  
  # Decoder
  decode_id = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="decode_input_word_ids")
  decode_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                      name="decode_input_mask")
  decode_seg_id = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, 
                                        name="decode_input_segment_ids")
  decode_input = [decode_id, decode_mask, decode_seg_id]
  _, decode_embedding = bert_layer(decode_input)
  decode_LSTM = tf.keras.layers.LSTM(n_hidden_unit, return_state=True, return_sequences=True)
  decode_output, _, _ = decode_LSTM(decode_embedding, initial_state=[state_h, state_c])
  
  output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size, activation='softmax'))(decode_output[:, : -1, :])
  model = tf.keras.models.Model([encode_input, decode_input], output)
  
  return model

model = seq2seq_model(vocab_size=vocab_size, max_seq_length=max_seq_len, n_hidden_unit=n_hidden_unit)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decode_input_word_ids (InputLay [(None, 128)]        0                                            
__________________________________________________________________________________________________
decode_input_mask (InputLayer)  [(None, 128)]        0                                            
__________________________________________________________________________________________________
decode_input_segment_ids (Input [(None, 128)]        0                                            
__________________________________________________________________________________________________
encode_input_word_ids (InputLay [(None, 128)]        0                                            
______________________________________________________________________________________________

In [24]:
# Train the model
batch_size = 8
steps_per_epoch = len(train_encode_ids) // batch_size
train_data = batch_iter(train_encode_ids, train_encode_masks, train_encode_segs, train_decode_ids, train_decode_masks, train_decode_segs, train_labels, batch_size, max_seq_len, vocab_size)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# train_data = [train_encode_ids, train_encode_masks, train_encode_segs, train_decode_ids, train_decode_masks, train_decode_ids]
validation_data = [test_encode_ids, test_encode_masks, test_encode_segs, test_decode_ids, test_decode_masks, test_decode_ids]
model.fit_generator(train_data, epochs=3, verbose=1, steps_per_epoch=128, validation_data=(validation_data, to_categorical(test_labels, vocab_size)))
# model.fit(train_data, to_categorical(train_labels, vocab_size), epochs=5)
tf.keras.models.save_model(model, 'generate_model.h5')

model = tf.keras.models.load_model('generate_model.h5', custom_objects={'KerasLayer':hub.KerasLayer})
blank_answer = np.array([['[CLS]'] + ['[PAD]'] * 127] * len(test_encode_ids))

test_encode_ids = tf.convert_to_tensor(test_encode_ids, dtype=tf.int32, name='input_word_ids')
test_encode_masks = tf.convert_to_tensor(test_encode_masks, dtype=tf.int32, name='input_mask')
test_encode_segs = tf.convert_to_tensor(test_encode_segs, dtype=tf.int32, name='input_type_ids')

test_decode_ids = tf.convert_to_tensor(test_decode_ids, dtype=tf.int32, name='input_word_ids')
test_decode_masks = tf.convert_to_tensor(test_decode_masks, dtype=tf.int32, name='input_mask')
test_decode_segs = tf.convert_to_tensor(blank_answer, dtype=tf.int32, name='input_type_ids')

test_data = [test_encode_ids, test_encode_masks, test_encode_segs, test_decode_ids, test_decode_masks, test_decode_segs]
predictions = model.predict(test_data, verbose=1)

# print(len(vocab_i2w))
with open("output.txt", "w") as f:
    for p in predictions:
        p = np.argmax(p, axis=1)
        p = [str(pp) for pp in p] 
        f.write('\t'.join(p) + '\n')

(8, 128)
Epoch 1/3


KeyboardInterrupt: ignored